# This kernel based on the kernel [One-Hot, Stratified, Logistic Regression](https://www.kaggle.com/bustam/one-hot-stratified-logistic-regression) 

## This kernel use tuning from the my kernel [Logistic Regression with my tuning](https://www.kaggle.com/vbmokin/logistic-regression-with-my-tuning)

In [ ]:
import numpy as np
import pandas as pd
import os
import scipy
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# Import and explore data

In [ ]:
train = pd.read_csv('/kaggle/input/cat-in-the-dat/train.csv')
test = pd.read_csv('/kaggle/input/cat-in-the-dat/test.csv')

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.nunique()

In [ ]:
train.isna().sum()

Now we create column classes defined by their type of data.

In [ ]:
bin_cols = [col  for col in train.columns.values if col.startswith('bin')]
nom_cols = [col  for col in train.columns.values if col.startswith('nom')]
ord_cols = [col  for col in train.columns.values if col.startswith('ord')]

We need to encode the data. The next functio converts the original data to integer using map from keys to values. The keys are generated by a sorted array of the unique strings of the column, the values by an array of integers from 0 to the length of the keys, or by a manually sorted array.

## Ordinal Columns

In [ ]:
def ord_to_num(df, col):
    keys=np.sort(df[col].unique())
    values=np.arange(len(keys))
    map = dict(zip(keys, values))
    df[col] = df[col].replace(map)

In [ ]:
ord_to_num(test, 'ord_3')
ord_to_num(train, 'ord_3')

In [ ]:
ord_to_num(train, 'ord_4')
ord_to_num(test,'ord_4')

In [ ]:
ord_to_num(train, 'ord_5')
ord_to_num(test,'ord_5')

In [ ]:
keys_ord_1=train.ord_1.unique()
keys_ord_1

In [ ]:
values_ord_1=[3,4,0,1,2]

In [ ]:
map_ord_1 = dict(zip(keys_ord_1, values_ord_1))
map_ord_1

In [ ]:
train['ord_1'] = train['ord_1'].replace(map_ord_1)
test['ord_1'] = test['ord_1'].replace(map_ord_1)

In [ ]:
keys_ord_2=train.ord_2.unique()
keys_ord_2

In [ ]:
values_ord_2=[1,3,5,4,0,2]

In [ ]:
map_ord_2 = dict(zip(keys_ord_2, values_ord_2))
map_ord_2

In [ ]:
train['ord_2'] = train['ord_2'].replace(map_ord_2)
test['ord_2'] = test['ord_2'].replace(map_ord_2)

In [ ]:
train[ord_cols].head()

In [ ]:
train[ord_cols].nunique()

We are going to stratify ord_4 and ord_5.

In [ ]:
train['ord_4_band'] = pd.qcut(train['ord_4'], 6)
bands=train.ord_4_band.unique()
keys_bands=np.sort(bands)
values_bands=np.arange(len(keys_bands))
map_bands = dict(zip(keys_bands, values_bands))

In [ ]:
train['ord_4_band'] = train['ord_4_band'].replace(map_bands)
test['ord_4_band']=pd.cut(test.ord_4,pd.IntervalIndex(keys_bands))

In [ ]:
test['ord_4_band'] = test['ord_4_band'].replace(map_bands)
test.ord_4_band.head()

In [ ]:
train['ord_5_band'] = pd.qcut(train['ord_5'], 6)
bands=train.ord_5_band.unique()
keys_bands=np.sort(bands)
values_bands=np.arange(len(keys_bands))
map_bands = dict(zip(keys_bands, values_bands))

In [ ]:
train['ord_5_band'] = train['ord_5_band'].replace(map_bands)
test['ord_5_band']=pd.cut(test.ord_5,pd.IntervalIndex(keys_bands))

In [ ]:
test['ord_5_band'] = test['ord_5_band'].replace(map_bands)
test.ord_5_band.head()

## Nominal Columns

In [ ]:
train[nom_cols].nunique()

In [ ]:
test[nom_cols].nunique()

We see that the are columns with different number of values on the train and test set.  We will
change those values with the same one

In [ ]:
for col in ["nom_7", "nom_8", "nom_9"]:
    train_vals = set(train[col].unique())
    test_vals = set(test[col].unique())
   
    ex=train_vals ^ test_vals
    if ex:
        train.loc[train[col].isin(ex), col]="x"
        test.loc[test[col].isin(ex), col]="x"

In [ ]:
train[nom_cols].nunique()

In [ ]:
test[nom_cols].nunique()

In [ ]:
train[train.nom_7=='x']

In [ ]:
train=train[train.nom_7!='x']

In [ ]:
train[train.nom_7=='x']

Now we label encode the nom_cols.

In [ ]:
labelEnc=LabelEncoder()

In [ ]:
for col in nom_cols:
    train[col]=labelEnc.fit_transform(train[col])
    test[col]=labelEnc.fit_transform(test[col])


In [ ]:
train[nom_cols].head()

## Binary Columns

Now we take a look at the bin_cols.

In [ ]:
train[bin_cols].head()

We need to encode bin_3 and bin_4.

In [ ]:
for col in ['bin_3', 'bin_4']:
    train[col]=labelEnc.fit_transform(train[col])
    test[col]=labelEnc.fit_transform(test[col])


In [ ]:
test[bin_cols].head()

## One-Hot Encoding

We will one-hot encode all features.

In [ ]:
X_temp=train.drop('target', axis=1)
Y=train.target

In [ ]:
from sklearn.mixture import GaussianMixture

In [ ]:
gm = GaussianMixture(n_components=4)
gm.fit(X_temp)

In [ ]:
X_temp['Gaussian_Mixture']=gm.predict(X_temp)

In [ ]:
test['Gaussian_Mixture']=gm.predict(test)

In [ ]:
X_oh_temp=pd.get_dummies(X_temp.drop('id', axis=1), columns=X_temp.drop('id', axis=1).columns, drop_first=True, sparse=True)

In [ ]:
X_oh=scipy.sparse.csr_matrix(X_oh_temp.values)

In [ ]:
X_oh

We do the same for the test set.

In [ ]:
test_oh_temp=pd.get_dummies(test.drop('id', axis=1), columns=test.drop('id', axis=1).columns, drop_first=True, sparse=True)

In [ ]:
test_oh_temp.shape

In [ ]:
test_oh=scipy.sparse.csr_matrix(test_oh_temp.values)

# Logistic Regression

We split X_oh into train and validation sets.

In [ ]:
X_train, X_valid, Y_train, Y_valid = train_test_split(X_oh, 
                                                      Y, 
                                                      test_size = 0.20,
                                                      random_state=42)


In [ ]:
lr=LogisticRegression(C=0.15, solver="lbfgs", tol=0.00005, max_iter=10000)  

lr.fit(X_train, Y_train)

In [ ]:
y_pred_lr=lr.predict_proba(X_valid)

In [ ]:
roc_auc_score(Y_valid.values, y_pred_lr[:,1])

In [ ]:
lr.fit(X_oh, Y)

In [ ]:
y_pred=lr.predict_proba(X_oh)
roc_auc_score(Y.values, y_pred[:,1])

In [ ]:
test_pred_lr=lr.predict_proba(test_oh)

In [ ]:
output_dict = {'id': test.id,
                       'target': test_pred_lr[:,1]}


output = pd.DataFrame(output_dict, columns = ['id', 'target'])
output.head(10)

In [ ]:
output.to_csv('submission_One_Hot.csv', index=False)